In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.preprocessing import QuantileTransformer

import xgboost
from xgboost import XGBRegressor    
from xgboost import XGBRegressor as model

def count1(df):
    result=[]
    index=df.index.tolist()
    for i in index:
        x=df.loc[i].sum()
        result.append(x-1)
    return result

def vcl_tr(df):
    df_=df.loc[:, 'VCL1':'VCL16']
    result=[]
    index=df_.index.tolist()
    for i in index:
        x=df_.loc[i].sum()
        if(x>=9):
            result.append(1)
        else:
            result.append(0)
    train['VCL_M']=result
    
def vcl_te(df):
    df_=df.loc[:, 'VCL1':'VCL16']
    result=[]
    index=df_.index.tolist()
    for i in index:
        x=df_.loc[i].sum()
        if(x>=9):
            result.append(1)
        else:
            result.append(0)
    test['VCL_M']=result

def count0(df):
    result=[]
    index=df.index.tolist()
    for i in index:
        x=df.loc[i].sum()
        result.append(x-1)
    return result


def q_tr(df):
    df_=df.loc[:, 'Q1':'Q26']
    result=[]
    index=df_.index.tolist()
    for i in index:
        x=df_.loc[i].tolist()
        re=0
        for j in x:
            re+=((j-1)/4)*100
        result.append(re/(len(x)-1))
    train['Q_m']=result
    
def q_te(df):
    df_=df.loc[:, 'Q1':'Q26']
    result=[]
    index=df_.index.tolist()
    for i in index:
        x=df_.loc[i].tolist()
        re=0
        for j in x:
            re+=((j-1)/4)*100
        result.append(re/(len(x)-1))
    test['Q_m']=result
    
def question_nan(df,col):
    train_q0=df[df[col]==0.0]
    train_q1=df[df[col]!=0.0]
    test_=train_q0
    #결과 없는 test
    train_=train_q1
    #결과 들어가있는 train
    test_=test_.drop(col,axis=1)
    y = train_[col]
    x= train_.drop(col,axis =1)
    model = XGBRegressor(objective='reg:squarederror',learning_rate=0.1,max_depth = 4, n_estimators = 1000)
    model.fit(x, y)
    pred = model.predict(test_)      
    pred=pred.round(0)
    pred=pred.tolist()
    for i in range(len(pred)):
        if(pred[i]<=0):
            pred[i]=1
    index=test_.index.tolist()
    for i in range(len(index)):
        df.at[index[i],col]=pred[i]


In [2]:
sample = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
Question_te=test.loc[:, 'Q1':'Q26']

In [3]:
country_map = {}

for i, c in enumerate(train['country'].unique()):
    country_map[c] = i
    

In [4]:
train['country'] = train['country'].map(country_map)
test['country'] = test['country'].map(country_map) #country열 Mapping

In [5]:
test = test.fillna(0)
train = train.fillna(0)#test 결측치 0으로 채우기

In [6]:
vcl_tr(train)
vcl_te(test)

In [7]:

train=train.drop(train.loc[:,'VCL1':'VCL16'],axis=1)
test=test.drop(test.loc[:,'VCL1':'VCL16'],axis=1)

In [8]:
q_tr(train)
q_te(test)

In [ ]:


col=Question_te.columns.tolist()
col

for i in col:
    question_nan(train,i)
    
for i in col:
    question_nan(test,i)
    

In [ ]:

train=train.drop('index',axis=1)
test=test.drop('index',axis=1)

In [ ]:

x_train = train.drop('nerdiness',axis=1)
x_train=x_train.loc[:, 'Q1':'Q26']
y_train = train['nerdiness']
test=test=test.loc[:, 'Q1':'Q26']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42, n_jobs=-1)

from sklearn.model_selection import KFold, GridSearchCV #하이퍼파라미터 튜닝을 위한 불러오기
from sklearn.metrics import accuracy_score

model_param_grid = {'n_estimators' : [100, 1000, 3000]} # Default(100)대비 큰값으로 Dictionary구성

model_grid = GridSearchCV(model, param_grid = model_param_grid, scoring="accuracy",n_jobs=-1,verbose=1) #훈련 (장치 사양에 따라 시간이 걸릴 수 있음)
model_grid.fit(x_train, y_train)

In [ ]:

gcv=GridSearchCV(model, param_grid=model_param_grid, scoring='f1', n_jobs=-1)

In [ ]:

gcv.fit(x_train.values,y_train.values)
print('final params', gcv.best_params_)   # 최적의 파라미터 값 출력

model = RandomForestClassifier(n_estimators=3000, random_state=42, n_jobs=-1)

model.fit(x_train, y_train) #학습

In [ ]:

pred = model.predict_proba(test)[:,1] #proba를 통해 확률값으로 변환

In [ ]:
sample['nerdiness'] = pred

In [ ]:
sample['nerdiness']=sample['nerdiness'].round(3)
sample['nerdiness']=sample['nerdiness'].round(2)
sample['nerdiness']=sample['nerdiness'].round(1)
sample['nerdiness']=sample['nerdiness'].round(0)

In [ ]:
sample[sample['nerdiness']>=2]


In [ ]:
sample.to_csv('result8.6_2.csv',index=False)

In [ ]:
sample['nerdiness'].sum()
#19384.0

In [ ]:
test

In [ ]:
train